In [1]:
using Pkg
using Revise
Pkg.activate(joinpath(@__DIR__, "../../"))
using Globtim
using DynamicPolynomials, DataFrames
# Constants and Parameters
const n, a, b = 3, 1, 1
const scale_factor = a / b   # Scaling factor appears in `main_computation`, maybe it should be a parameter.
const delta, alpha = 0.5, 1 / 10  # Sampling parameters
const tol_l2 = 3e-4            # Define the tolerance for the L2-norm
f = Rastringin # Objective function

  Activating project at `~/globtim`


Rastringin (generic function with 1 method)

We set the number of samples used to generate the approximant. It is annoying that the error goes up while the degree has increased.

In [2]:
rand_center = [0.0, 0.0, 0,];
d = 10 # initial degree 
SMPL = 40 # Number of samples
TR = test_input(f, 
                dim = n,
                center=rand_center,
                GN=SMPL, 
                sample_range=scale_factor, 
                degree_max =d+4
                )
pol_cheb = Constructor(TR, d, basis=:chebyshev)
pol_lege = Constructor(TR, d, basis=:legendre);

@polyvar(x[1:n]); # Define polynomial ring 

current L2-norm: 8.892146854778785e-5
current L2-norm: 0.06280960975634996


Solve the system of partial derivatives using `Msolve`. We should allow for points slightly outside the domain. 

In [8]:
¡# df_cheb = solve_and_parse(pol_cheb, x, f, TR)
df_cheb = msolve_parser("tmp_output_GDJnbt8n.ms", f, TR)
sort!(df_cheb, :z, rev=false)


=== Starting MSolve Parser (dimension: 3) ===
Processed 125 points (0.005s)


Row,x1,x2,x3,z
,Float64,Float64,Float64,Float64
1,0.0,0.0,-2.79995e-16,-30.0
2,0.0,0.0,-2.64482e-16,-30.0
3,0.0,0.0,-1.70165e-16,-30.0
4,0.0,0.0,-1.37169e-16,-30.0
5,0.0,0.0,-1.30726e-16,-30.0
6,0.0,0.0,-1.22857e-16,-30.0
7,0.0,0.0,-1.21412e-16,-30.0
8,0.0,0.0,-1.16693e-16,-30.0
9,0.0,0.0,-1.1623e-16,-30.0


In [4]:
using StaticArrays
using GLMakie
include("LevelSetViz.jl")
using .LevelSetViz

In [5]:
grid = scale_factor * generate_grid(3, 100)  # 3D grid
values = map(f, grid); # Prepare level set data for specific level

In [9]:
fig = create_level_set_visualization(f, grid, df_cheb, (-30.0, 30.))

In [7]:
# GLMakie.closeall()